# Cuaderno 10: Problema de asignación óptima

A menudo las variables y restricciones de un modelo de programación lineal (entera) dependen de índices que representan elementos de conjuntos finitos.

Un ejemplo clásico de un problema que puede ser formulado como un programa lineal entero es el **problema de asignación óptima** (*assignment problem*).

Se tienen dados un conjunto $I$ de personas y un conjunto $J$ de tareas. Se ha medido la afinidad de cada persona $i \in I$ para desempeñar cada tarea $j \in J$, expresando este valor a través de una calificación $c_{ij}$, donde una calificación más alta indica una mayor afinidad (es decir, una mayor destreza en el desempeño de la tarea por parte de la persona). Se desea asignar cada tarea a una persona, de tal forma que se maximice la suma de las calificaciones correspondientes a las asignaciones realizadas. Ninguna tarea puede ser asignada a más de una persona, y ninguna persona puede tener más de una tarea asignada.

El problema de encontrar una asignación óptima puede ser formulado como el siguiente programa lineal entero.

\begin{align*}
&\max \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \\ 
&\mbox{s.r.}\\
& \sum_{j \in J} x_{ij} \leq 1, \quad \forall i \in I, \\
& \sum_{i \in I} x_{ij} \leq 1, \quad \forall j \in J, \\
& x_{ij} \in \{0, 1\}, \quad \forall i \in I, j \in J.
\end{align*}

Cada variable binaria de decisión $x_{ij}$ indica si una persona $i \in I$ ha sido asignada a una tarea $j \in J$. La función objetivo mide la calificación total de todas las asignaciones realizadas. 

La primera familia de restricciones expresa que ninguna persona puede tener más de una tarea asignada. Observar que existe una restricción por cada persona.

La segunda familia de restricciones expresa que ninguna tarea puede ser asignada a más de una persona. Observar que existe una restricción por cada tarea.

Vas a implementar este modelo usando la interfaz Python de Gurobi. Primero es necesario importar módulo `gurobipy`:

In [14]:
from gurobipy import *

Consideraremos para este ejemplo un conjunto $I$ de cuatro personas y un conjunto $J$ de cuatro tareas. Definimos estos conjuntos como tuplas:

In [15]:
# tupla con el conjunto ordenado de personas
I = ('Pablo', 'Carmen', 'Sofía', 'Pedro')
# tupla con el conjunto ordenado de tareas
J = ('base de datos', 'módulo de reportes', 'análisis estadístico', 'modelo de optimización')

print ('I:= {}'.format(I))
print ('J:= {}'.format(J))

I:= ('Pablo', 'Carmen', 'Sofía', 'Pedro')
J:= ('base de datos', 'módulo de reportes', 'análisis estadístico', 'modelo de optimización')


Las calificaciones de todas las posibles asignaciones se almacenan en un diccionario `calif` indexado por los pares ordenados $(i,j) \in I \times J$: 

In [16]:
# puntajes de afinidad persona-tarea
calif = { 
    ('Pablo', 'base de datos') : 9, 
    ('Pablo', 'módulo de reportes') : 10, 
    ('Pablo', 'análisis estadístico') : 8, 
    ('Pablo', 'modelo de optimización') : 9, 
    ('Carmen', 'base de datos') : 10, 
    ('Carmen', 'módulo de reportes') : 9, 
    ('Carmen', 'análisis estadístico') : 8, 
    ('Carmen', 'modelo de optimización') : 8, 
    ('Sofía', 'base de datos') : 10, 
    ('Sofía', 'módulo de reportes') : 8, 
    ('Sofía', 'análisis estadístico') : 10, 
    ('Sofía', 'modelo de optimización') : 9, 
    ('Pedro', 'base de datos') : 9, 
    ('Pedro', 'módulo de reportes') : 8, 
    ('Pedro', 'análisis estadístico') : 8, 
    ('Pedro', 'modelo de optimización') : 10} 


El siguiente paso es crear el objeto modelo:

In [17]:
m = Model('problema-asignacion')

Para crear las variables, podemos usar el método `addVars`. Este método permite crear una familia de variables indexadas por uno o más conjuntos iterables. En nuestro caso, las variables $x_{ij}$ están indexadas por todas las combinaciones de personas $i \in I$ y tareas $j \in J$. Por lo tanto, los dos primeros argumentos de la función serán `I` y `J`:

In [21]:
# creamos variables x indexadas por I x J
x = m.addVars(I, J, vtype = GRB.BINARY, name="x")
print(type(x))
m.update()
#print(x)
#print([x[i,j].varName for i in I for j in J])

<class 'gurobipy.tupledict'>


Los demás argumentos de la función son los mismos que los argumentos de `addVar`. El argumento `name` en este caso no fija el nombre de la variable, sino un prefijo que es completado con las posibles combinaciones de los índices.

Definimos ahora la función objetivo. El método `prod` del objeto `x` se usa implementar el sumatorio $\sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij}$:

In [22]:
# la función objetivo es la suma de c_ij x_ij sobre i en I y j en J
m.setObjective(x.prod(calif, '*', '*'), GRB.MAXIMIZE)

Definimos ahora cada una de las familias de restricciones empleando el método `addConstrs`:

1. Cada persona puede tener asignada máximo una tarea: el primer parámetro de `addConstrs` es una expresión generadora que depende de la variable `i`, la misma que toma los valores de la tupla `I`. Para cada valor específico de `i`, el método `sum` de `x` se encarga de calcular la suma $\sum_{j \in J} x_{ij}$. El parámetro `asig_i` fija un nombre para la familia de restricciones. El nombre de cada restricción individual se genera automáticamente apendizando un sufijo con los elementos de `I`.

In [24]:
# Cada persona puede tener asignada máximo una tarea
cI = m.addConstrs((x.sum(i, '*') <= 1 for i in I), "asig_i") 
m.update()
print(type(cI))
print([cI[i].getAttr('ConstrName') for i in I]) 

<class 'gurobipy.tupledict'>
['asig_i[Pablo]', 'asig_i[Carmen]', 'asig_i[Sofía]', 'asig_i[Pedro]']


2. Cada tarea puede estar asignada máximo a una persona: utilizamos nuevamente el método `addConstrs`, cuyo primer parámetro es una expresión generadora que depende de la variable `j`, la misma que toma los valores de la tupla `J`. Para cada valor específico de `j`, el método `sum` de `x` se encarga de calcular la suma $\sum_{i \in I} x_{ij}$. El parámetro `asig_j` fija un nombre para la familia de restricciones.

In [26]:
# Cada tarea puede estar asignada máximo a una persona
m.addConstrs((x.sum('*', j) <= 1 for j in J), "asig_j") 

{'base de datos': <gurobi.Constr *Awaiting Model Update*>,
 'módulo de reportes': <gurobi.Constr *Awaiting Model Update*>,
 'análisis estadístico': <gurobi.Constr *Awaiting Model Update*>,
 'modelo de optimización': <gurobi.Constr *Awaiting Model Update*>}

Finalmente, resolvemos el modelo:

In [9]:
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x359e125e
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 38.0000000
Presolve time: 0.02s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 4.000000e+01, 1 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      40.0000000   40.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.24 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 40 38 

Optimal solution found (tolerance 1.00e-04)
Best objec

Terminada la optimización, podemos mostrar las soluciones. Notar que se usa la sintaxis `x[i,j]` para acceder a la variable $x_{ij}$, pues `x` es un diccionario cuyas claves son pares ordenados del conjunto $I \times J$:

In [10]:
for i in I:
    for j in J:
        if x[i, j].x >= 0.99:
            print("Tarea {}: asignada a {}.".format(j, i))

Tarea módulo de reportes: asignada a Pablo.
Tarea base de datos: asignada a Carmen.
Tarea análisis estadístico: asignada a Sofía.
Tarea modelo de optimización: asignada a Pedro.


## Código completo

Reproducimos a continuación el código completo del ejemplo:

In [1]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de asignación óptima
# EPN (2020)

from gurobipy import *
try:
    # tupla con el conjunto ordenado de personas
    I = ('Pablo', 'Carmen', 'Sofía', 'Pedro')
    # tupla con el conjunto ordenado de tareas
    J = ('base de datos', 'módulo de reportes', 'análisis estadístico', 'modelo de optimización')

    # puntajes de afinidad persona-tarea
    calif = { 
        ('Pablo', 'base de datos') : 9, 
        ('Pablo', 'módulo de reportes') : 10, 
        ('Pablo', 'análisis estadístico') : 8, 
        ('Pablo', 'modelo de optimización') : 9, 
        ('Carmen', 'base de datos') : 10, 
        ('Carmen', 'módulo de reportes') : 9, 
        ('Carmen', 'análisis estadístico') : 8, 
        ('Carmen', 'modelo de optimización') : 8, 
        ('Sofía', 'base de datos') : 10, 
        ('Sofía', 'módulo de reportes') : 8, 
        ('Sofía', 'análisis estadístico') : 10, 
        ('Sofía', 'modelo de optimización') : 9, 
        ('Pedro', 'base de datos') : 9, 
        ('Pedro', 'módulo de reportes') : 8, 
        ('Pedro', 'análisis estadístico') : 8, 
        ('Pedro', 'modelo de optimización') : 10} 

    m = Model('problema-asignacion')

    # creamos variables x indexadas por I x J
    x = m.addVars(I, J, vtype = GRB.BINARY, name="x")
    
    # la función objetivo es la suma de c_ij x_ij sobre i en I y j en J
    m.setObjective(x.prod(calif, '*', '*'), GRB.MAXIMIZE)

    # Restricciones
    # Cada persona puede tener asignada máximo una tarea
    cI = m.addConstrs((x.sum(i, '*') <= 1 for i in I), "asig_i") 

    # Cada tarea puede estar asignada máximo a una persona
    m.addConstrs((x.sum('*', j) <= 1 for j in J), "asig_j") 

    # Resolver el modelo
    m.optimize()

    # Mostrar la solución
    for i in I:
        for j in J:
            if x[i, j].x >= 0.99:
                print("Tarea {}: asignada a {}.".format(j, i))
            
except GurobiError as e:
    print('Código de error ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se encontró un error de atributo')

Using license file /Users/ltorres/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x359e125e
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 38.0000000
Presolve time: 0.01s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 4.000000e+01, 1 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      40.0000000   40.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.10 seconds
Thread count was 4 (of 4 available p